___

<div style="text-align: center;">
  <span style="font-family: 'Playfair Display', serif; font-size: 24px; font-weight: bold;">
    Constructing a Knowledge Graph and Creating Embeddings for Price Group Prediction
  </span>
</div>

___

In this notebook, we focus on constructing a Knowledge Graph and generating embeddings for predicting price groups. The main steps include:

- RDF Triplets: We have mapped the cleaned data to the RDF schema, and now we convert it into RDF triples (subject-predicate-object format).
- Embedding Generation: Using the TransE model, we generate embeddings from the RDF triples to capture the relationships within the knowledge graph.
- Model Training: The embeddings are used as input features to train a Multi-Layer Perceptron (MLP) model for price group prediction.

In [ ]:
#!pip install torchkge

In [ ]:
import os
import torch
import pickle
import numpy as np
import pandas as pd
from rdflib import Graph
from torch.optim import Adam
from tqdm.autonotebook import tqdm
from torchkge.models import TransEModel
from torchkge.utils import MarginLoss, DataLoader
from torchkge.data_structures import KnowledgeGraph
from torchkge.sampling import BernoulliNegativeSampler

In [ ]:
# Load the graph from TTL file
g = Graph()
g.parse("./../../data/explotation_zone/RDFGraph_Model_emb.ttl", format="ttl")

# Extract triples in order to create the Knowledge graph
print('Creating the triples...')
triples = []
for s, p, o in g:
    triples.append((str(s), str(p), str(o)))

directory = './../data/explotation_zone'  
os.makedirs(directory, exist_ok=True)
file_path = os.path.join(directory, 'RDFTriples.txt')

# Save triples into a file
with open(file_path, 'w') as f:
    for triple in triples:
        f.write("\t".join(triple) + "\n")
print('Done!')
print()

print('Creating the Knowledge graph...')
# Convert into DataFrame and reorganize col to torchKGE format
data = pd.DataFrame(triples, columns=['from', 'rel', 'to'])
data = data[['from', 'to', 'rel']]

In [ ]:
data[data['rel'] == 'http://example.org/apartment/price_discretized']

Split between train and test:

We split the data between train and test. Since we only want to predict the price of AirBNB apartments, only triples corresponding to the price relation are going to be evaluated. The model will still learn the relations between all nodes, including the nodes for which the later model will be predicting it's price group (although this relations has been erased)

In [ ]:
test = data[data['rel'] == 'http://example.org/apartment/price_discretized'].sample(frac = 0.2)
train = data.drop(test.index)
test_y = test['to']
test_X = test.drop(columns = ['to'])

Build the training graph:

In [ ]:
kg_train = KnowledgeGraph(df=train)

The model will be giving an output of 100 dimension embedding and will be trained with the parameters that can be seen in the code. The selected model is a TransE projection model. This selection has been made due to it's simplicity and therefore faster training.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train_model(kg_train, kg_val=None, emb_dim=100, n_epochs=1000, b_size=32768, lr=0.0004, margin=0.5):
    """
    Trains an embedding model using torchKGE with a custom training loop.
        :param kg_train: KnowledgeGraph, the training knowledge graph
        :param kg_val: KnowledgeGraph, the validation knowledge graph (optional)
        :param emb_dim: int, the dimension of the embeddings
        :param n_epochs: int, the number of epochs to train
        :param b_size: int, the batch size for training
        :param lr: float, the learning rate
        :param margin: float, the margin value for the TransE loss function
    """
    # Define the embedding model
    model = TransEModel(emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2')

    # Define the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)

    # Define the loss function
    criterion = MarginLoss(margin)

    # Define the negative sampler and dataloader
    sampler = BernoulliNegativeSampler(kg_train)
    dataloader = DataLoader(kg_train, batch_size=b_size, use_cuda=device)

    # Training loop
    c = 0
    iterator = tqdm(range(n_epochs), unit='epoch')
    for epoch in iterator:
        running_loss = 0.0
        for i, batch in enumerate(dataloader):
            h, t, r = batch[0], batch[1], batch[2]
            n_h, n_t = sampler.corrupt_batch(h, t, r)

            optimizer.zero_grad()

            # Forward + backward + optimize
            pos, neg = model(h, t, r, n_h, n_t)
            loss = criterion(pos, neg)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        iterator.set_description(
            'Epoch {} | mean loss: {:.5f}'.format(epoch + 1, running_loss / len(dataloader)))
        
        c += 1

        model.normalize_parameters()

In [ ]:
model = train_model(kg_train)

The following cell is used as a mere checkpoint for the notebook

In [ ]:
with open('train.pkl', 'rb') as outp:
    train = pickle.load(outp)

with open('kg_train.pkl', 'rb') as outp:
    kg_train = pickle.load(outp)

with open('test_X.pkl', 'rb') as outp:
    test_X = pickle.load(outp)

with open('test_y.pkl', 'rb') as outp:
    test_y = pickle.load(outp)

with open('model.pkl', 'rb') as outp:
    model = pickle.load(outp)

In [ ]:
test_X

<div class="alert alert-block alert-info" style="color: #01571B; background-color: #C8E5C6;">

Now, and using batch-processing to not overload the memory, we will get the embedding for each apartment in the training and test datasets separately. This will be later loaded in a torch tensor to train a future network.

Train

In [ ]:
train_embedding_list = []
batch_size = 1000  # Adjust according to memory capacity

filtered_items = train[train['rel'] == 'http://example.org/apartment/price_discretized']['from']

for start in range(0, len(filtered_items), batch_size):
    batch = filtered_items[start:start + batch_size]
    batch_embeddings = []
    for item in batch:
        i = kg_train.ent2ix[item]
        embedding = model.get_embeddings()[0][i]
        if isinstance(embedding, np.ndarray):
            batch_embeddings.append(embedding)
        else:
            batch_embeddings.append(np.array(embedding))  
    
    # Convert batch_embeddings to a NumPy array
    batch_embeddings = np.stack(batch_embeddings)
    
    # Save intermediate results to disk
    np.save(f'batch_{start}.npy', batch_embeddings)

Test

In [ ]:
train_embedding_list = []
batch_size = 500  # Adjust according to memory capacity

filtered_items = test_X['from']

for start in range(0, len(filtered_items), batch_size):
    batch = filtered_items[start:start + batch_size]
    batch_embeddings = []
    for item in batch:
        i = kg_train.ent2ix[item]
        embedding = model.get_embeddings()[0][i]
        if isinstance(embedding, np.ndarray):
            batch_embeddings.append(embedding)
        else:
            batch_embeddings.append(np.array(embedding)) 
    
    # Convert batch_embeddings to a NumPy array
    batch_embeddings = np.stack(batch_embeddings)
    
    # Save intermediate results to disk
    np.save(f'./test_X/batch_{start}.npy', batch_embeddings)